In [50]:
import cv2
import dlib
import math
import pickle
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.svm import SVC

%matplotlib inline

In [51]:
face_detector = dlib.get_frontal_face_detector()
landmark_predictor = dlib.shape_predictor("./dlib/shape_predictor_68_face_landmarks.dat")

In [53]:
with open('./dataset/ck.pikle','rb') as f:
    ck_data, ck_label = pickle.load(f)
    
face_landmarks_train = [] 
face_label_train = []
face_landmarks_test = []
face_label_test = []

ck_data_train, ck_data_test, ck_label_train, ck_label_test = train_test_split(ck_data, np.asarray(ck_label), test_size=0.25, random_state=42)
counter = 1

for face, label in zip(ck_data_train, ck_label_train):
    face = np.array(face)
    faces = face_detector(face.reshape(128,128), 1)
    for k,d in enumerate(faces):
        shape = landmark_predictor(face.reshape(128,128), d)
        
        xlist = [float(shape.part(i).x) for i in range(68)]
        ylist = [float(shape.part(i).y) for i in range(68)]
        xmean = np.mean(xlist)
        ymean = np.mean(ylist)
        xcentral = xlist - xmean
        ycentral = ylist - ymean
            
        landmarks_vectorised = []
        for x, y, w, z in zip(xcentral, ycentral, xlist, ylist):
            landmarks_vectorised.append(w)
            landmarks_vectorised.append(z)
            
            meannp = np.asarray((ymean,xmean))
            coornp = np.asarray((z,w))
            dist = np.linalg.norm(coornp-meannp)
            
            landmarks_vectorised.append(dist)
            landmarks_vectorised.append((math.atan2(y, x)*360)/(2*math.pi))
        
        face_landmarks_train.append(landmarks_vectorised)
        face_label_train.append(label)
    
for face, label in zip(ck_data_test, ck_label_test):
    face = np.array(face)
    faces = face_detector(face.reshape(128,128), 1)
    
    for k,d in enumerate(faces):
        shape = landmark_predictor(face.reshape(128,128), d)
        
        xlist = [float(shape.part(i).x) for i in range(68)]
        ylist = [float(shape.part(i).y) for i in range(68)]
        xmean = np.mean(xlist)
        ymean = np.mean(ylist)
        xcentral = xlist - xmean
        ycentral = ylist - ymean
        
        landmarks_vectorised = []
        for x, y, w, z in zip(xcentral, ycentral, xlist, ylist):
            landmarks_vectorised.append(w)
            landmarks_vectorised.append(z)
            
            meannp = np.asarray((ymean,xmean))
            coornp = np.asarray((z,w))
            dist = np.linalg.norm(coornp-meannp)
            
            landmarks_vectorised.append(dist)
            landmarks_vectorised.append((math.atan2(y, x)*360)/(2*math.pi))
        
        face_landmarks_test.append(landmarks_vectorised)
        face_label_test.append(label)
        

In [54]:
target_classes = ["Neutral", "Angry", "Disgust", "Fear", "Happy", "Sad", "Surprise"]

clf = SVC(kernel='linear', probability=True, tol=1e-3)

clf = clf.fit(face_landmarks_train, face_label_train)
ck_label_predict = clf.predict(face_landmarks_test)

print(classification_report(ck_label_test, ck_label_predict, target_names=target_classes))
print(confusion_matrix(ck_label_test, ck_label_predict, labels=range(7)))
print(accuracy_score(ck_label_test, ck_label_predict))

              precision    recall  f1-score   support

     Neutral       0.80      0.67      0.73        12
       Angry       0.77      0.91      0.83        11
     Disgust       0.91      0.91      0.91        11
        Fear       1.00      0.50      0.67         6
       Happy       0.94      1.00      0.97        15
         Sad       0.56      0.83      0.67         6
    Surprise       1.00      0.95      0.98        21

   micro avg       0.87      0.87      0.87        82
   macro avg       0.85      0.82      0.82        82
weighted avg       0.88      0.87      0.86        82

[[ 8  2  0  0  0  2  0]
 [ 0 10  0  0  0  1  0]
 [ 1  0 10  0  0  0  0]
 [ 1  0  1  3  1  0  0]
 [ 0  0  0  0 15  0  0]
 [ 0  1  0  0  0  5  0]
 [ 0  0  0  0  0  1 20]]
0.8658536585365854


In [55]:
len(face_landmarks_train)

245